In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
!pip install transformers
!pip install sacremoses
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from torch.utils.data import TensorDataset
torch.cuda.empty_cache()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.0 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 
     |████████████████████████████████| 182 kB 67.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 7.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=55cd16465472db51a3bf2f3324c991c81310b067e7f0d4f6c6cb652212f60822
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
englishToRussian = "facebook/wmt19-en-ru"
tokenizerForward = FSMTTokenizer.from_pretrained(englishToRussian)
modelForward = FSMTForConditionalGeneration.from_pretrained(englishToRussian)

russianToEnglish = "facebook/wmt19-ru-en"
tokenizerBackward = FSMTTokenizer.from_pretrained(russianToEnglish)
modelBackward = FSMTForConditionalGeneration.from_pretrained(russianToEnglish)
torch.cuda.empty_cache()

Downloading:   0%|          | 0.00/639k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/776k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/639k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

In [ ]:
modelForward = modelForward.to('cuda')
modelBackward = modelBackward.to('cuda')
torch.cuda.empty_cache()

In [ ]:
# creating (query, target) DataFrame from dataset
import os

from zipfile import ZipFile
from tqdm.notebook import tqdm
with ZipFile('/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/stance-data-all-annotations.zip') as z:
    z.extractall()
dataDir = './data-all-annotations'
import pandas as pd
with open(os.path.join(dataDir, 'trainingdata-all-annotations.txt'), encoding='windows-1252') as f:
    lines = f.readlines()
queries = []
targets = []
stances = []
maxLen = 0
for line in tqdm(lines[1:]):
    temp = line.split('\t')
    maxLen = max(maxLen, len(temp[2]))
    queries.append(temp[2].lower())
    targets.append(temp[1].lower())
    stances.append(temp[3])

print(queries[0], targets[0], maxLen)
df= pd.DataFrame(zip(queries, targets, stances), columns=['Query', 'Target', 'Stance'])

possible_stance = df.Stance.unique()
stance_dict = {}
for index, possible_stance in enumerate(possible_stance):
    stance_dict[possible_stance] = index
df['Stance'] = df.Stance.replace(stance_dict)

possible_target = df.Target.unique()
target_dict = {}
for index, possible_target in enumerate(possible_target):
    target_dict[possible_target] = index
df['Target'] = df.Target.replace(target_dict)
print(df.Target.unique(), df.Stance.unique(), len(df))

  0%|          | 0/2814 [00:00<?, ?it/s]

dear lord thank u for all of ur blessings forgive my sins lord give me strength and energy for this busy day ahead #blessed #hope #semst atheism 142
[0 1 2 3 4] [0 1 2] 2814


In [ ]:
print(stance_dict)
#print(target_dict)

{'AGAINST': 0, 'FAVOR': 1, 'NONE': 2}


In [ ]:
# Open Procon20

usingProcon = True

if usingProcon:
    proconfile_train = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/procon_final.csv', encoding='utf8', errors='backslashreplace')
    procon_train = pd.read_csv(proconfile_train, sep=',', header=0)

    # Create a copy and backtranslate the tweets
    df2 = procon_train.copy()

    possible_target2 = df2.Target.unique()
    target_dict2 = {}
    for index, possible_target in enumerate(possible_target2):
        target_dict2[possible_target] = index
    df2['Target'] = df2.Target.replace(target_dict2)
    print(df2.Target.unique(), df2.Stance.unique(), len(df2))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144] [0 1] 5984


SEMEVAL

In [ ]:
import torch
tokenizerForward = FSMTTokenizer.from_pretrained(englishToRussian, 
                                          do_lower_case=True)
# we can use max_length = 280 i.e. max characters allowed in a tweet.                                          
encoded_data = tokenizerForward.batch_encode_plus(
    df.Query.values, 
    add_special_tokens=True, 
    return_attention_mask=False, 
    padding='max_length', 
    max_length=280, 
    truncation=True,
    return_tensors='pt'
)
targets = torch.tensor(df.Target.values)
stances = torch.tensor(df.Stance.values)


input_ids = encoded_data['input_ids']

dataset = TensorDataset(input_ids, targets, stances)

tokenizerBackward = FSMTTokenizer.from_pretrained(russianToEnglish, 
                                          do_lower_case=True)
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader = DataLoader(dataset, 
                              sampler=RandomSampler(dataset), 
                              batch_size=batch_size)
torch.cuda.empty_cache()

PROCON

In [ ]:
if usingProcon:
  tokenizerForward2 = FSMTTokenizer.from_pretrained(englishToRussian, 
                                            do_lower_case=True)
  # we can use max_length = 280 i.e. max characters allowed in a tweet.                                          
  encoded_data2 = tokenizerForward2.batch_encode_plus(
      df2.Query.values, 
      add_special_tokens=True, 
      return_attention_mask=False, 
      padding='max_length', 
      max_length=3097, 
      truncation=True,
      return_tensors='pt'
  )
  targets2 = torch.tensor(df2.Target.values)
  stances2 = torch.tensor(df2.Stance.values)


  input_ids2 = encoded_data2['input_ids']

  dataset2 = TensorDataset(input_ids2, targets2, stances2)

  tokenizerBackward2 = FSMTTokenizer.from_pretrained(russianToEnglish, 
                                            do_lower_case=True)

  batch_size2 = 8

  dataloader2 = DataLoader(dataset2, 
                                sampler=RandomSampler(dataset2), 
                                batch_size=batch_size2)
  torch.cuda.empty_cache()

In [ ]:
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision

def tokenize(text):
    tokenized_data = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokenized_data.append(word)
    return tokenized_data

def get_bleu_score(reference, candidate):
    reference = [tokenize(reference)]
    candidate = tokenize(candidate)
    bleu_score = float(modified_precision(reference, candidate, n=1))
    return bleu_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


FOR SEMEVAL

In [ ]:
# I suggest 0.75 for threshold
print('Enter bleu score threshold value : ')
threshold = 0.75
data = pd.DataFrame(columns=df.columns)
modelForward.eval()
modelBackward.eval()
for batch in tqdm(dataloader):
    targets = batch[1]
    stances = batch[2]
    queries = tokenizerBackward.batch_decode(batch[0])
    batch = tuple(b.to('cuda') for b in batch)
    with torch.no_grad():  
        russianEncoded = modelForward.generate(batch[0])
        englishEncoded = modelBackward.generate(russianEncoded)
        outputs = tokenizerBackward.batch_decode(englishEncoded)
        for query, output, target, stance in zip(queries, outputs, targets, stances):
            o = str(output).strip('<pad>').strip('</s>')
            if get_bleu_score(query, o) > threshold:
                data.loc[len(data.index)] = [o, int(target), int(stance)] # adding the new data to the dataframe
#data.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/train_russian_backtranslated.csv')
#print('Augmented data file stored')

Enter bleu score threshold value : 


  0%|          | 0/88 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


FOR PROCON

In [ ]:
if usingProcon:
  print('Enter bleu score threshold value : ')
  threshold = 0.75
  data2 = pd.DataFrame(columns=df.columns)
  modelForward.eval()
  modelBackward.eval()
  for batch in tqdm(dataloader2):
      targets = batch[1]
      stances = batch[2]
      queries = tokenizerBackward2.batch_decode(batch[0])
      batch = tuple(b.to('cuda') for b in batch)
      with torch.no_grad():  
          russianEncoded = modelForward.generate(batch[0])
          englishEncoded = modelBackward.generate(russianEncoded)
          outputs = tokenizerBackward2.batch_decode(englishEncoded)
          for query, output, target, stance in zip(queries, outputs, targets, stances):
              o = str(output).strip('<pad>').strip('</s>')
              if get_bleu_score(query, o) > threshold:
                  data2.loc[len(data2.index)] = [o, int(target), int(stance)] # adding the new data to the dataframe

Enter bleu score threshold value : 


  0%|          | 0/748 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
# replacing target ids with target values
target_dict = {v : k for k, v in target_dict.items()}
data['Target'] = data.Target.replace(target_dict)
if usingProcon:
  target_dict2 = {v : k for k, v in target_dict2.items()}
  data2['Target'] = data2.Target.replace(target_dict2)

In [ ]:
#Combine Procon and Semeval!
if usingProcon:
  final_backtranslated = pd.concat([data,data2], ignore_index=True)
  final_backtranslated.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/Both/russian_backtranslated.csv')
else: 
  final_backtranslated = data.copy()
  final_backtranslated.to_csv(f'/content/drive/MyDrive/CSE 573: Stance Detection /Datasets/SemEval-2016/modified/russian_backtranslated.csv')

In [ ]:
stance_dict